In [2]:
from transformers import AutoTokenizer
import transformers
from datasets import load_dataset
import torch
import re

In [18]:
def get_sequences(prompt, tokenizer, llm_pipeline):
    sequences = llm_pipeline(
        prompt
        , num_return_sequences=1
        , eos_token_id = tokenizer.eos_token_id
    )
    return sequences

def get_pipeline(model):
    pipeline = transformers.pipeline(
        "text-generation"
        , model = model
        , torch_dtype = torch.float16
        , device_map = "auto"
    )
    return pipeline

def get_json_format(text):
    dialogues = []
    lines = [line.strip() for line in text.split('\n') if line.strip()]

    situation = []

    for line in lines:
        if not line.startswith("Situation"):
            situation.append(line.strip())

    return situation

In [12]:
# mistral_path = "/kaggle/input/mistral/pytorch/7b-v0.1-hf/1"
# mistral_tokenizer = AutoTokenizer.from_pretrained(mistral_path)
# mistral_pipeline = get_pipeline(mistral_path)

llama_path = "/kaggle/input/llama-2/pytorch/7b-chat-hf/1"
llama_tokenizer = AutoTokenizer.from_pretrained(llama_path)
llama_pipeline = get_pipeline(llama_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
prompt = f'''
            Describe five situations between two friends on different occasions.
            The topic is {topics[0]}.
            '''
seq = get_sequences(prompt, llama_tokenizer, llama_pipeline)
seq = seq[0]["generated_text"].replace(prompt, "")

In [21]:
sits = get_json_format(seq)[:5]

In [ ]:
prompt = f'''
        Create a dialogue between 2 friends, given this situation: {sits[0]}
'''
seq = get_sequences(prompt, llama_tokenizer, llama_pipeline)
seq = seq[0]["generated_text"].replace(prompt, "")
seq